In [9]:
import mne
%pip install mne_bids
%pip install --upgrade mne_bids

from mne_bids import BIDSPath, print_dir_tree, write_raw_bids, make_dataset_description, update_sidecar_json

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [10]:
#Allows us to work with file paths
import os.path as op
#Makes file path handling simpler
from pathlib import Path

In [11]:
#testing the reiterating run through
import mne
%pip install mne_bids
%pip install --upgrade mne_bids

from mne_bids import BIDSPath, print_dir_tree, write_raw_bids, make_dataset_description, update_sidecar_json
#Allows us to work with file paths
import os.path as op
#Makes file path handling simpler
from pathlib import Path

location = Path(r'c:\N8_internship_code\Motor_Imaging_Dataset')

#Change the file path to your data's location
data_dir = location
print_dir_tree(data_dir, max_depth=1)


children = [child for child in data_dir.iterdir()]
children

dir_number = 5
files = [file for file in children[dir_number].iterdir()]
file_path = files[1]
file_path

data = mne.io.read_raw_edf(file_path)
data


data_dir = location
print_dir_tree(data_dir, max_depth=1)

children = [child for child in data_dir.iterdir()]
children


from datetime import datetime, timezone
from dateutil.relativedelta import relativedelta
from mne.transforms import Transform

# Load your Excel data
df = pd.read_excel(r'c:\N8_internship_code\N8 tester sheet.xlsx')

mne.channels.get_builtin_montages()
builtin_montages = mne.channels.get_builtin_montages(descriptions=True)

for montage_name, montage_description in builtin_montages:
    print(f"{montage_name}: {montage_description}")

Note: you may need to restart the kernel to use updated packages.

|Motor_Imaging_Dataset\
|--- 64_channel_sharbrough-old.png
|--- 64_channel_sharbrough.pdf
|--- 64_channel_sharbrough.png
|--- ANNOTATORS
|--- RECORDS
|--- SHA256SUMS.txt
|--- wfdbcal
|--- S001\
|--- S002\
|--- S003\
|--- S004\
|--- S005\
|--- S006\
|--- S007\
|--- S008\
|--- S009\
|--- S010\
|--- S011\
|--- S012\
|--- S013\
|--- S014\
|--- S015\
|--- S016\
|--- S017\
|--- S018\
|--- S019\
|--- S020\
|--- S021\
|--- S022\
|--- S023\
|--- S024\
|--- S025\
|--- S026\
|--- S027\
|--- S028\
|--- S029\
|--- S030\
|--- S031\
|--- S032\
|--- S033\
|--- S034\
|--- S035\
|--- S036\
|--- S037\
|--- S038\
|--- S039\
|--- S040\
|--- S041\
|--- S042\
|--- S043\
|--- S044\
|--- S045\
|--- S046\
|--- S047\
|--- S048\
|--- S049\
|--- S050\
|--- S051\
|--- S052\
|--- S053\
|--- S054\
|--- S055\
|--- S056\
|--- S057\
|--- S058\
|--- S059\
|--- S060\
|--- S061\
|--- S062\
|--- S063\
|--- S064\
|--- S065\
|--- S066\
|--- S067\
|--- S068\
|-

NameError: name 'pd' is not defined

In [ ]:

folders = [p_folder for p_folder in location.iterdir() if p_folder.is_dir()]

for idx, participant_folder in enumerate(folders):
    # Get value from a specific column at the same index
    participant_id = df.loc[idx, 'participant_id']    
    last_name = df.loc[idx, 'last_name']
    first_name = df.loc[idx, 'first_name']
    middle_name = df.loc[idx, 'middle_name']
    age = df.loc[idx, 'age']
    sex = df.loc[idx, 'sex']
    hand = df.loc[idx, 'hand']
    weight = df.loc[idx, 'weight']
    height = df.loc[idx, 'height']

    years = (age)
    # Generates the (approx) birthdate of the participant based on the measurement date and age
    data.set_meas_date(datetime(2015, 6, 7, tzinfo= timezone.utc))
    recording_date = data.info["meas_date"]
    Birthdate = recording_date - relativedelta(years)

    data.info["subject_info"] = {
        "id": idx,
        "his_id": participant_id,
        "last_name": last_name,
        "first_name": first_name,
        "middle_name": middle_name,
        "birthday": Birthdate,
        "sex": int(sex),
        "hand": int(hand),
        "weight": weight,
        "height": height,
    }

    data.info["device_info"] = {
        "type": "EEG",
        "model": "12-channel EEG",
        "serial": 33456423,
        "site": "yes"
        }
    

    data.info["line_freq"] = 50


    data.info["description"] = "a resting state dataset"

    data.info["dev_head_t"] = Transform("meg", "head")

    data.info["experimenter"] = "John Doe"

    
    bids_root = op.join(data_dir.parent, "iterate_test")

    my_montage = mne.channels.make_standard_montage("biosemi64")
    
    file_name = participant_folder.name

    #Edit this information to match your data
    subject_id = file_name
    task = "task1"

    bids_path = BIDSPath(subject=subject_id, task=task, root=bids_root)
    write_raw_bids(data, bids_path, overwrite=True, allow_preload=True, format="EDF")


Writing 'c:\N8_internship_code\iterate_test\README'...
Writing 'c:\N8_internship_code\iterate_test\participants.tsv'...
Writing 'c:\N8_internship_code\iterate_test\participants.json'...
The provided raw data contains annotations, but you did not pass an "event_id" mapping from annotation descriptions to event codes. We will generate arbitrary event codes. To specify custom event codes, please pass "event_id".
Used Annotations descriptions: ['T0']
Writing 'c:\N8_internship_code\iterate_test\sub-S001\eeg\sub-S001_task-task1_events.tsv'...
Writing 'c:\N8_internship_code\iterate_test\sub-S001\eeg\sub-S001_task-task1_events.json'...
Writing 'c:\N8_internship_code\iterate_test\dataset_description.json'...


Writing 'c:\N8_internship_code\iterate_test\sub-S001\eeg\sub-S001_task-task1_eeg.json'...
Writing 'c:\N8_internship_code\iterate_test\sub-S001\eeg\sub-S001_task-task1_channels.tsv'...
Copying data files to sub-S001_task-task1_eeg.edf
Reading 0 ... 9759  =      0.000 ...    60.994 secs...
Writing 'c:\N8_internship_code\iterate_test\sub-S001\sub-S001_scans.tsv'...
Wrote c:\N8_internship_code\iterate_test\sub-S001\sub-S001_scans.tsv entry with eeg\sub-S001_task-task1_eeg.edf.
Writing 'c:\N8_internship_code\iterate_test\participants.tsv'...
Writing 'c:\N8_internship_code\iterate_test\participants.json'...
The provided raw data contains annotations, but you did not pass an "event_id" mapping from annotation descriptions to event codes. We will generate arbitrary event codes. To specify custom event codes, please pass "event_id".
Used Annotations descriptions: ['T0']
Writing 'c:\N8_internship_code\iterate_test\sub-S002\eeg\sub-S002_task-task1_events.tsv'...
Writing 'c:\N8_internship_code\ite

C:\Users\Ariana Williams\AppData\Local\Temp\ipykernel_27816\1452292734.py:67: RuntimeWarning: Converting data files to EDF format
  write_raw_bids(data, bids_path, overwrite=True, allow_preload=True, format="EDF")
C:\Users\Ariana Williams\AppData\Local\Temp\ipykernel_27816\1452292734.py:67: RuntimeWarning: Converting data files to EDF format
  write_raw_bids(data, bids_path, overwrite=True, allow_preload=True, format="EDF")
C:\Users\Ariana Williams\AppData\Local\Temp\ipykernel_27816\1452292734.py:67: RuntimeWarning: Converting data files to EDF format
  write_raw_bids(data, bids_path, overwrite=True, allow_preload=True, format="EDF")
C:\Users\Ariana Williams\AppData\Local\Temp\ipykernel_27816\1452292734.py:67: RuntimeWarning: Converting data files to EDF format
  write_raw_bids(data, bids_path, overwrite=True, allow_preload=True, format="EDF")


Writing 'c:\N8_internship_code\iterate_test\dataset_description.json'...
Writing 'c:\N8_internship_code\iterate_test\sub-S005\eeg\sub-S005_task-task1_eeg.json'...
Writing 'c:\N8_internship_code\iterate_test\sub-S005\eeg\sub-S005_task-task1_channels.tsv'...
Copying data files to sub-S005_task-task1_eeg.edf
Reading 0 ... 9759  =      0.000 ...    60.994 secs...
Writing 'c:\N8_internship_code\iterate_test\sub-S005\sub-S005_scans.tsv'...
Wrote c:\N8_internship_code\iterate_test\sub-S005\sub-S005_scans.tsv entry with eeg\sub-S005_task-task1_eeg.edf.
Writing 'c:\N8_internship_code\iterate_test\participants.tsv'...
Writing 'c:\N8_internship_code\iterate_test\participants.json'...
The provided raw data contains annotations, but you did not pass an "event_id" mapping from annotation descriptions to event codes. We will generate arbitrary event codes. To specify custom event codes, please pass "event_id".
Used Annotations descriptions: ['T0']
Writing 'c:\N8_internship_code\iterate_test\sub-S006\

C:\Users\Ariana Williams\AppData\Local\Temp\ipykernel_27816\1452292734.py:67: RuntimeWarning: Converting data files to EDF format
  write_raw_bids(data, bids_path, overwrite=True, allow_preload=True, format="EDF")
C:\Users\Ariana Williams\AppData\Local\Temp\ipykernel_27816\1452292734.py:67: RuntimeWarning: Converting data files to EDF format
  write_raw_bids(data, bids_path, overwrite=True, allow_preload=True, format="EDF")
C:\Users\Ariana Williams\AppData\Local\Temp\ipykernel_27816\1452292734.py:67: RuntimeWarning: Converting data files to EDF format
  write_raw_bids(data, bids_path, overwrite=True, allow_preload=True, format="EDF")
C:\Users\Ariana Williams\AppData\Local\Temp\ipykernel_27816\1452292734.py:67: RuntimeWarning: Converting data files to EDF format
  write_raw_bids(data, bids_path, overwrite=True, allow_preload=True, format="EDF")
C:\Users\Ariana Williams\AppData\Local\Temp\ipykernel_27816\1452292734.py:67: RuntimeWarning: Converting data files to EDF format
  write_raw_bi

Wrote c:\N8_internship_code\iterate_test\sub-S010\sub-S010_scans.tsv entry with eeg\sub-S010_task-task1_eeg.edf.


KeyError: 10

In [ ]:
folders = [participant for participant in location.iterdir() if participant.is_dir()]
indexes = list(range(len(folders)))
print(indexes)



[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108]


In [ ]:
import pandas as pd
from pathlib import Path
sheet = Path(r'c:\N8_internship_code\N8 tester sheet.xlsx')

df = pd.read_excel(sheet, sheet_name=0) # can also index sheet by name or fetch all sheets
data_info_list = df['participant_id'].tolist()
print(data_info_list)


['s001', 's002', 's003', 's004', 's005', 's006', 's007', 's008', 's009', 's010']


's002'

In [ ]:
for idx, folder in enumerate(folders):
    participant_id = data_info_list[idx]
    print(f"Folder: {folder.name} → Participant ID: {participant_id}")

Folder: S001 → Participant ID: s001
Folder: S002 → Participant ID: s002
Folder: S003 → Participant ID: s003
Folder: S004 → Participant ID: s004
Folder: S005 → Participant ID: s005
Folder: S006 → Participant ID: s006
Folder: S007 → Participant ID: s007
Folder: S008 → Participant ID: s008
Folder: S009 → Participant ID: s009
Folder: S010 → Participant ID: s010


IndexError: list index out of range

In [ ]:
for participant_folder in location.iterdir():
    if not participant_folder.is_file():
        folders = [f for f in location.iterdir() if f.is_dir()]

        # Load your Excel data
        df = pd.read_excel(r'c:\N8_internship_code\N8 tester sheet.xlsx')

        for idx, folder in enumerate(folders):
            # Get value from a specific column at the same index
            participant_id = df.loc[idx, 'participant_id']     # or: df['age'].iloc[idx]
            first_name = df.loc[idx, 'first_name']  
            last_name = df.loc[idx, 'last_name']
            middle_name = df.loc[idx, 'middle_name']
            age = df.loc[idx, 'age']
            sex = df.loc[idx, 'sex']
            hand = df.loc[idx, 'hand']
            weight = df.loc[idx, 'weight']
            height = df.loc[idx, 'height']

            
            print(f"""
                Participant folder: {folder.name}
                ID          : {participant_id}
                Name        : {first_name} {middle_name} {last_name}
                Age         : {age}
                Sex         : {sex}
                Handedness  : {hand}
                Weight (kg) : {weight}
                Height (cm) : {height}
                """)


                Participant folder: S001
                ID          : s001
                Name        : Olivia Rae Nguyen
                Age         : 23
                Sex         : 1
                Handedness  : 1
                Weight (kg) : 60
                Height (cm) : 165
                

                Participant folder: S002
                ID          : s002
                Name        : Liam James Patel
                Age         : 29
                Sex         : 1
                Handedness  : 2
                Weight (kg) : 74
                Height (cm) : 178
                

                Participant folder: S003
                ID          : s003
                Name        : Ava Marie Kim
                Age         : 35
                Sex         : 2
                Handedness  : 2
                Weight (kg) : 68
                Height (cm) : 170
                

                Participant folder: S004
                ID          : s004
          

KeyError: 10

In [ ]:
from pathlib import Path
import pandas as pd
from datetime import datetime, timezone
from dateutil.relativedelta import relativedelta

# Set your root path
location = Path(r"C:\N8_internship_code\source_data")

# Get only folders (participants)
folders = [participant for participant in location.iterdir() if participant.is_dir()]

# Load Excel sheet once
sheet = Path(r'c:\N8_internship_code\N8 tester sheet.xlsx')
df = pd.read_excel(sheet, sheet_name=0)

# Ensure the sheet has a column that can match the folder names (e.g., 'participant_id')
# Assuming folders are named like "sub-s001" and Excel has 'participant_id' like "s001"
df['folder_name'] = df['participant_id'].apply(lambda x: f"sub-{x}")

# Loop through each participant folder
for folder in folders:
    folder_name = folder.name

    # Match row in DataFrame
    row = df[df['folder_name'] == folder_name]

    age = int(row['age'].values[1])  # Get age from Excel
    print(f"{folder_name} → Age: {age}")

    # Simulated EEG data object placeholder (replace with real one)
    # data = mne.io.read_raw_edf(folder / "filename.edf")  # <- Example

    # Set measurement date and compute birthdate
    meas_date = datetime(2015, 6, 7, tzinfo=timezone.utc)
    # data.set_meas_date(meas_date)
    birthdate = meas_date - relativedelta(years=age)

    print(f"Estimated birthdate for {folder_name}: {birthdate.date()}")
    print(age)

IndexError: index 1 is out of bounds for axis 0 with size 0

In [ ]:
file_name = participant_folder.name
file_name

In [ ]:
#Change the file path to your data's location
data_dir = Path(r"C:\N8_internship_code\source_data")
print_dir_tree(data_dir, max_depth=1)

In [ ]:
children = [child for child in data_dir.iterdir()]
children

In [ ]:
#Change this to match the file number
dir_number = 0
files = [file for file in children[dir_number].iterdir()]

In [ ]:
file_path = files[0]
file_path

In [ ]:
data = mne.io.read_raw_eeglab(file_path)

In [ ]:
data

In [ ]:
#specify line frequency
data.info["line_freq"] = 50

In [ ]:
bids_root = op.join(data_dir.parent, "bids_example")
bids_root

In [ ]:
#Edit this information ot match your data
subject_id = "001"
task = "rest"

bids_path = BIDSPath(subject=subject_id, task=task, root=bids_root)
write_raw_bids(data, bids_path, overwrite=True, allow_preload=True, format="EEGLAB")

In [ ]:
#creating a dataset description JSON file (separate to other data entry types) [for people who want a better description of their dataset than previously provided] 
# #{will overwrite any existing dataset_description.json file in the root of the BIDS directory}
make_dataset_description(
    path=bids_root,
    name="EEGManyLabs Resting State Study", 
    hed_version="1",
    dataset_type='raw',
    data_license="CCO",
    authors=["Ariana Williams", "Daniel Brady"],
    acknowledgements=None,
    how_to_acknowledge="Cite (Williams et al., 2025) when using this dataset",
    funding=["The NHS", "The Uk government"],
    ethics_approvals="Ethical approval was granted by the University of Leeds School of Psychology Ethics committee (12345 2025)",
    references_and_links="https://mne.tools/mne-bids/stable/whats_new_previous_releases.html",
    doi="doi:https://doi.org/10.1016/j.tins.2017.02.004",
    generated_by=[
        {
            "Name": "MNE-BIDS",
            "Version": "0.14",
            "Description": "Used to convert MEG data into BIDS format."
        },
        {
            "Name": "MNE-Python",
            "Version": "1.6.1",
            "Description": "Used for MEG preprocessing and analysis."
        }
    ],
    source_datasets=[
        {
            "URL": "https://example.com/source_dataset",
            "DOI": "10.1234/example.doi",
        }],
            overwrite=True,
            verbose=True)

In [ ]:
#updating a specific parameter in a JSON [SIDECAR]
json_file = Path(r'C:\N8_internship_code\bids_example\sub-001\eeg\sub-001_task-rest_eeg.json').absolute()
# Create BIDSPath using the JSON file's directory as root
root = (r'C:\N8_internship_code\bids_example')

bids_path1 = BIDSPath(subject='001', task='rest',
                     suffix='eeg', extension='.json', datatype='eeg',
                     root=root)

entries = {'EEGReference' : "FCz",
           'SamplingFrequency': 1000.0,
           'PowerLineFrequency': 50.0,
           'SoftwareFilters': {
                "Anti-aliasing filter":{
                "half-amplitude cutoff (Hz)": 500,
                "Roll-off": "6dB/Octave"
                }
                },
            'TaskName': "Resting State",
            'TaskDescription': "Participants were asked to rest with their eyes closed for 5 minutes.",
            'Instructions': "Please close your eyes and relax. Do not think about anything in particular.",
            'CogAtlasID': "", 
            'CogPOID': "",    
            'Manufacturer': "Brain Products GmbH",
            'ManufacturersModelName':"Brain Products actiCHamp",
            'CapManufacturer': "EasyCap",
            'CapManufacturersModelName': "EasyCap 64-channel cap",
            'SoftwareVersions' : {
                'MNE-BIDS': "1.0.0",
                'MNE-Python': "1.6.1"
                },
            'DeviceSerialNumber': "123456789",
            'EEGChannelCount': 64,
            'ECGChannelCount': 2,
            'EMGChannelCount': 0,
            'EOGChannelCount': 0,
            'MiscChannelCount': 0,
            'TriggerChannelCount': 0,
            'RecordingDuration': 600,
            'RecordingType': "Continuous",
            'EpochLength': 2.0,
            'EEGGround':"Fpz", 
            'HeadCircumference': 58.0,
            'EEGPlacementScheme': "10 percent system",
            'HardwareFilters':{
               "ADC's decimation filter (hardware bandwidth limit)":{
                "-3dB cutoff point (Hz)": 480,
                "Filter order sinc response": 5
                }
                },
            'SubjectArtefactDescription': "No known artifacts were present during the recording.",
            'InstitutionName': "University of Leeds",
            'InstitutionAddress': "Leeds, UK", 
            'InstitutionalDepartmentName': "School of Psychology", 
            'ElectricalStimulation': "",
            'ElectricalStimulationParameters': ""}
           
update_sidecar_json(bids_path1, entries, verbose=True)

In [ ]:
#updating a specific parameter in a JSON [COORDSYSTEM]
json_file = Path(r'C:\N8_internship_code\bids_example\sub-001\eeg\sub-001_space-CapTrak_coordsystem.json').absolute()
# Create BIDSPath using the JSON file's directory as root
root = Path(r'C:\N8_internship_code\bids_example')


bids_path1 = BIDSPath(subject='001', task=None,
                     suffix='coordsystem', extension='.json', datatype='eeg',
                     root=root, space='CapTrak')

entries = {'EEGCoordinateSystem':"CapTrak",
           'EEGCoordinateUnits':"m",
           'EEGCoordinateSystemDescription': "CapTrak is a 3D digitization system that provides accurate spatial coordinates for EEG electrode placement.",
           'FiducialsDescription': "Fiducials are anatomical landmarks used to align the EEG cap with the participant's head. The fiducials used in this study were Nasion, Left Preauricular, and Right Preauricular.",
           'FiducialsCoordinates': {
                'Nasion': [0.0, 0.0, 0.0],
                'Left Preauricular': [-0.1, 0.0, 0.0],
                'Right Preauricular': [0.1, 0.0, 0.0]
           },
           'FiducialsCoordinateSystem': "CapTrak",
            'FiducialsCoordinateUnits': "m",
            'FiducialsCoordinateSystemDescription': "CapTrak fiducials are defined in a 3D coordinate system that aligns with the participant's head.",
            'AnatomicalLandmarkCoordinates': {
                'Nasion': [0.0, 0.0, 0.0],
                'Left Preauricular': [-0.1, 0.0, 0.0],
                'Right Preauricular': [0.1, 0.0, 0.0]
            },
            'AnatomicalLandmarkCoordinateSystem': "CapTrak",
            'AnatomicalLandmarkCoordinateUnits': "m",
            'AnatomicalLandmarkCoordinateSystemDescription': "CapTrak anatomical landmarks are defined in a 3D coordinate system that aligns with the participant's head."
            }       
update_sidecar_json(bids_path1, entries, verbose=True)

In [ ]:
#updating a specific parameter in a JSON [EVENTS]
json_file = Path(r'C:\N8_internship_code\bids_example\sub-001\eeg\sub-001_task-rest_events.json').absolute()
# Create BIDSPath using the JSON file's directory as root
root = (r'C:\N8_internship_code\bids_example')


bids_path1 = BIDSPath(subject='001', task='rest',
                     suffix='events', extension='.json', datatype='eeg',
                     root=root)

entries = {'Onset':{'Description': "Onset time of the event in seconds relative to the start of the recording.", 'Units': "seconds"},
        'Duration':{'Description': "Duration of the event in seconds.", 'Units': "Seconds"},
        'TrialType':{'LongName':"Event Category", 
                     'Description': "Indicator of the type of action that is expected.", 
                     'Levels': {
                        'Start': "A red square appears on the screen to indicate the start of a trial.",
                        'Stop': "A green square appears on the screen to indicate the end of a trial"}
                     },
        'ResponseTime':{'Description': "Time taken by the participant to respond to the event in seconds.", 'Units': "seconds"},
        'HED': {'musc': "EMG-artifact"},
        'Stim File': {'Description': "File containing the stimulus presentation information for the event.", 'FileType': "CSV", 'FilePath': "sub-001_task-rest_stim.csv"},
        'Channel': {'Description': "Channel used to record the event.", 'Type': "Trigger"}
        }       
update_sidecar_json(bids_path1, entries, verbose=True)

In [ ]:
#code for adding information to the participants.json file
#updating a specific parameter in a JSON
json_file = Path(r'C:\N8_internship_code\bids_example\participants.json').absolute()
# Create BIDSPath using the JSON file's directory as root
root = (r'C:\N8_internship_code\bids_example')


bids_path1 = BIDSPath(subject=None, task=None,
                     suffix='participants', extension='.json', datatype=None,
                     root=root)

entries = {'Partiipant ID':{'Description': "Unique Participant identifier"}, 
           'Species': {'Description': "Species of the participant"}, 
           'Age': {'Description': "Age of the participant at time of testing", 'Units': "Years"}, 
           'Sex': {'Description': "Biological sex of the participant", 
                   'Levels': {
                       "F":'Female',
                       "M":'Male'
                   }
                },
            'Handedness': {'Description': "Handedness of the participant",
                        'Levels': {
                            "R": "Right-handed",
                            "L": "Left-handed",
                            "A": "Ambidextrous"
                        }
                },
            'Strain': {'Description': "Strain of the participant, if applicable"},
            'Strain RRID': {'Description': "Strain RRI ID of the participant, if applicable"},
        }       
update_sidecar_json(bids_path1, entries, verbose=True)

This is the extended code, the first bit should be automatically done in the pipeline, I can edit this to just update some parameters

Necessary:
>
    1. Participant ID 

Recommended:
>
    2. Species
    3. Age
    set_meas_date()
    4. Sex
    5. Handedness
    6. Strain
    7. Strain RRID
>
Optional:
>
    - Additional participant information may be included to further bolster your metadata.

In [ ]:
data.info.keys()

In [ ]:
from datetime import datetime, timezone
from dateutil.relativedelta import relativedelta

# Generates the (approx) birthdate of the participant based on the measurement date and age

data.set_meas_date(datetime(2015, 6, 7, tzinfo= timezone.utc))

recording_date = data.info["meas_date"]

Birthdate = recording_date - relativedelta(years=30)
print(Birthdate)


In [ ]:
data.info["subject_info"] = {
    "id": 1,
    "his_id": "sub-001",
    "last_name": "Doe",
    "first_name": "John",
    "middle_name": "A",
    "birthday": Birthdate,
    "sex": 2,
    "hand": 1,
    "weight": 70.0,
    "height": 175.0,
}

In [ ]:
data.info["subject_info"]


In [ ]:
data.get_montage()

In [ ]:
from datetime import datetime, timezone
from dateutil.relativedelta import relativedelta
from mne.transforms import Transform

data.info["device_info"] = {
    "type": _,
    "model": "_",
    "serial": "_",
    "site": "_"
    }



# Generates the (approx) birthdate of the participant based on the measurement date and age
data.set_meas_date(datetime(2015, 6, 7, tzinfo= timezone.utc))
recording_date = data.info["meas_date"]
Birthdate = recording_date - relativedelta(years=30)


data.info["subject_info"] = {
    "id": 1,
    "his_id": "sub-001",
    "last_name": "Doe",
    "first_name": "John",
    "middle_name": "A",
    "birthday": Birthdate,
    "sex": 2,
    "hand": 1,
    "weight": 70.0,
    "height": 175.0,
}

data.info["line_freq"] = 50

data.info["bads"] = ["C3", "F3" ]

data.info["description"] = "a restingstate dataset"

data.info["dev_head_t"] = Transform("meg", "head")

data.info["experimenter"] = "John Doe"

data.info["helium_info"] = {
    "he_level_raw": 20,
    "helium_level": 12,
    "orig_file_guid": "_",
    "meas_date": (datetime(2015, 6, 7, tzinfo= timezone.utc))
    }


In [ ]:
data.info["proj_name"] = "EEGManyLabs restingstate"

In [ ]:
data.info


In [ ]:
import pandas as pd

In [ ]:
participants_tsv = pd.read_csv(r'C:\N8_internship_code\bids_example\participants.tsv', sep='\t')

In [ ]:
education = ["High School"]  

participants_tsv["Education"] = education
participants_tsv.to_csv(r'C:\N8_internship_code\bids_example\participants.tsv', sep= '\t', index=False, na_rep='n/a')

In [ ]:
#way to iterate through participant IDs and print them
ID_list = []
for participant_id in participants_tsv["participant_id"]:
    ID_list.append(participant_id)
    print(ID_list)

In [ ]:
for _, row in participants_tsv.iterrows():
    participant_id = row["participant_id"]
    edu = row.get("education", "None")
    print(f"{participant_id}: Age = {edu}")


In [ ]:
import pandas as pd
participants_tsv = pd.read_csv(r'C:\N8_internship_code\bids_example\participants.tsv', sep='\t')
location = participants_tsv.loc[participants_tsv["participant_id"] == "sub-001"].index[0]
participants_tsv.loc[location, "Education"] = "None"
print (participants_tsv)

In [ ]:
import pandas as pd
participants_tsv = pd.read_csv(r'C:\N8_internship_code\bids_example\participants.tsv', sep='\t')
participants_tsv.loc[participants_tsv["participant_id"] == "sub-001", "Education"] = "none"
print(participants_tsv)
participants_tsv.to_csv(r'C:\N8_internship_code\bids_example\participants.tsv', sep= '\t', index=False, na_rep='n/a')

In [ ]:
print(participants_tsv)


In [ ]:
participants_tsv.drop([0, 0])
print(participants_tsv)

In [ ]:
new_row = pd.DataFrame([{"participant_id": "sub-002", "age": 30, "sex":"M"}])
participants_tsv = pd.concat([participants_tsv, new_row], ignore_index=True)
print(participants_tsv)
participants_tsv.to_csv(r'C:\N8_internship_code\bids_example\participants.tsv', sep= '\t', index=False, na_rep='n/a')

In [ ]:
participants_tsv = participants_tsv.drop(index=1)
print(participants_tsv)
participants_tsv.to_csv(r'C:\N8_internship_code\bids_example\participants.tsv', sep= '\t', index=False, na_rep='n/a')

In [ ]:
import pandas as pd
participants_tsv = pd.read_csv(bids_root / "participants.tsv", sep="\t")
for i, participant_id in enumerate(participant_ids):
    kwargs = dict(
        species="homo sapiens",
        age=age[i],
        sex=sex[i],
        handedness=hand[i],
        lab=lab,
        replication=study,
        old_id=oldids[i],
        tod=tod[i],
        education_year=edu_y[i],
        education_degree=edu_deg[i],
    )
    participants_tsv = _update_tsv(
        participants_tsv, "sub-" + participant_id, **kwargs
    )
    #** split dictionary
    #keyword arguments
participants_tsv = participants_tsv.astype(
    {"age": "Int64", "education_year": "Int64"}
)
participants_tsv.education_degree = (
    participants_tsv.education_degree.str.title()
)
participants_tsv[
    [
        "participant_id",
        "species",
        "age",
        "sex",
        "handedness",
        "tod",
        "education_year",
        "education_degree",
        "replication",
        "lab",
        "old_id",
    ]
].to_csv(
    BIDS_ROOT / "participants.tsv", sep="\t", index=False, na_rep="n/a"
)




In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
readme = op.join(bids_root, "README")
with open(readme, encoding="utf-8-sig") as fid:
    text = fid.read()
print(text)

In [ ]:
Possible code:

#mne bids code
# Get the electrode coordinates
testing_data = mne.datasets.testing.data_path()
captrak_path = op.join(testing_data, "montage", "captrak_coords.bvct")
montage = mne.channels.read_dig_captrak(captrak_path)

# Rename the montage channel names only for this example, because as said
# before, coordinate and EEG data were not actually collected together
# Do *not* do this for your own data.
montage.rename_channels(dict(zip(montage.ch_names, raw.ch_names)))

# "attach" the electrode coordinates to the `raw` object
# Note that this only works for some channel types (EEG/sEEG/ECoG/DBS/fNIRS)
raw.set_montage(montage)

# show the electrode positions
raw.plot_sensors()


In [ ]:
possible code:

Let’s pause and check that the information that we’ve written out to the sidecar files that describe our data is correct.
# Get the sidecar ``.json`` file
sidecar_json_bids_path = bids_path.copy().update(suffix="meg", extension=".json")
sidecar_json_content = sidecar_json_bids_path.fpath.read_text(encoding="utf-8-sig")
print(sidecar_json_content)

In [ ]:
# sample: the time point (in samples) where the event occurs
# 0- placeholder
# event_id: integer code identifying the event type
events = [
	[sample_1, 0, event_code_1],
  [sample_2, 0, event_code_2],
]  # shape (n_events, 3)

#dict uisng key:value pairs to display the event type and their associated code(integer)
event_id = {'stim1': 1, 'stim2': 2}

# Convert the events into annotations
annotations = mne.annotations_from_events(events, sfreq=raw.info['sfreq'], event_desc=event_id)

# Attach to raw
raw.set_annotations(annotations)

# Save to disk (optional)
raw.save('my_file_with_annotations.fif', overwrite=True)

NameError: name 'sample_1' is not defined

In [ ]:
import numpy as np

# Suppose your raw data is 1000 Hz and you want two events
sfreq = 1000  # sampling frequency

# At 2 seconds and 5 seconds
onsets_in_sec = [2, 5]
samples = [int(sfreq * t) for t in onsets_in_sec]

# Event codes: 1 = 'stim1', 2 = 'stim2'
event_codes = [1, 2]

# Build events array
events = np.array([[samples[i], 0, event_codes[i]] for i in range(len(samples))])

In [12]:
event_id = {'stim1': 1, 'stim2': 2}
annotations = mne.annotations_from_events(str(events), sfreq=sfreq, event_desc=event_id)
raw.set_annotations(annotations)

TypeError: Event names must be an instance of str or None, got <class 'int'> instead.